In [1]:
import open3d as o3d
import numpy as np
from PIL import Image

ModuleNotFoundError: No module named 'open3d'

In [15]:
K_ = np.matrix([[481.20, 0, 319.50, 0],
                [0, -480.00, 239.50, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])

K_inv = np.linalg.inv(K_)

# converting a pixel to a pointcloud point
def uvd2xyz(u, v, d):
    z = d / 5000
    xyz1 = z * K_inv @ np.array([u, v, 1, 1 / z]).transpose()
    return xyz1[0, 0], xyz1[0, 1], xyz1[0, 2]


# converting the images to a list of points and a list of corresponding colors
def get_points_and_colors(depths_img, colors_img):
    points = [uvd2xyz(u, v, depths_img.getpixel((u, v)))
              for u in range(depths_img.size[0]) for v in range(depths_img.size[1])]
    colors = [colors_img.getpixel((u, v))
              for u in range(colors_img.size[0]) for v in range(colors_img.size[1])]

    return np.array(points), np.array(colors) / 256


In [16]:
depths_img = Image.open('data/depth.png')
colors_img = Image.open('data/annot.png')
points, colors = get_points_and_colors(depths_img, colors_img)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

pcd.transform([[-1, 0, 0, 0],
               [0, -1, 0, 0],
               [0, 0, 1, 0],
               [0, 0, 0, -1]])
o3d.visualization.draw_geometries([pcd])